In [11]:
import os

In [12]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_data_dir: Path


In [15]:
from chicken_disease.constants import *
from chicken_disease.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzipped_data_dir= config.unzipped_data_dir
        )

        return data_ingestion_config


In [17]:
import os
import urllib.request as request
import zipfile 
from chicken_disease import logger
from chicken_disease.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file to the unzipped_data_dir
        Function returns None
        """
        unzip_path = self.config.unzipped_data_dir
        os.makedirs(unzip_path, exist_ok= True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
